# 伯努利朴素贝叶斯
以文字出没出现作为向量化准则(1为出现，0为未出现)

In [18]:
postingList=[['my','dog','has','flea','problems','help','please'],
            ['maybe','not','take','him','to','dog','park','stupid'],
            ['my','dalmation','is','so','cute','I','love','him'],
            ['stop','posting','stupid','worthless','garbage'],
            ['mr','licks','ate','my','steak','how','to','stop','him'],
            ['quit','buying','worthless','dog','food','stupid']]
#六行文本，每一行对应一个分类(1代表侮辱性文字,0代表正常言论)
classvec=[0,1,0,1,0,1]
import numpy as np

<span class="burk">实现步骤</span>

1.把所有出现的文字unique化，做成一个列表(可以把每个文字作为一个特征来看)

2.把每行文本对应这所有特征进行向量化，即出现为1，未出现为0

3.<span class="mark">计算每个类别下每个文字数目占该类别下总文字数目的比例(条件概率)</span>。再计算每个类别的出现的概率。

4.进行预测测试集，即把输入的文本中，每个文字出现概率相乘，但是为了避免概率为0(有些文字概率为0)，<span class="mark">所以需要调整下初始文字的数目为1,总数目增加2.</span>

同时由于概率相乘概率过小，所以将<span class="mark">第3步骤中的条件概率计算取</span>$In$,从而使数值不那么小，对比不同类别下的概率，选择大的概率作为分类标准

<span class="burk">注意：使用拉普拉斯平滑解决零概率问题；对乘积结果取自然对数避免下溢出问题，采用自然对数进行处理不会有任何损失。</span>条件概率一直相互乘积

In [19]:
#获得出现过的词语
def uniqueword(words):
    wordfeatures=set([])#使得加入的元素不会重复
    for i in words:
        wordfeatures=wordfeatures | set(i)#set(i)让加入的每行文本都是去重的,并集|作用是让行与行之间是去重的
    return list(wordfeatures)

In [20]:
#把文字转化为向量(即1为出现，0为未出现)
def wordvec(wordfeatures,row_word):
    p=np.zeros(len(wordfeatures))#初始化所有元素，都是未出现的
    for unit_word in row_word:
        if unit_word in wordfeatures:
            p[wordfeatures.index(unit_word)]=1#把出现过的词语变为1
    return p

In [66]:
#计算wordfeatures属于该类别的条件概率(用次数来做除法计算)和类别概率
def prob(words,category,wordfeatures,wordvec):
    p1=sum(category)/len(category)#得到侮辱性文字类别的概率
    #初始化
    p0w=np.ones(len(wordfeatures))#每个单词出现的数量为1，避免后续连乘时概率为0
    p0sum=2#种类0的文字总数,初始化
    p1w=np.ones(len(wordfeatures))
    p1sum=2#种类1的文字总数,初始化
    
    for i in range(len(category)):
        if category[i]==0:
            p0sum+=sum(wordvec[i])#存在这一类别，且出现了,sum,并且是词向量
            p0w+=wordvec[i]
        else:
            p1sum+=sum(wordvec[i])#存在这一类别，且出现了,sum,并且是词向量
            p1w+=wordvec[i]
    return np.log(p0w/p0sum),np.log(p1w/p1sum),p1#要取对数，避免值太小

In [67]:
wordfeatures=uniqueword(postingList)#获得出现的词语
#向量化所有文本
wordvecs=[]
for i in postingList:
    wordvecs.append(wordvec(wordfeatures,i))
#再次调用prob函数
p0word,p1word,p1=prob(postingList,classvec,wordfeatures,wordvecs)

[-1.87180218 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.15948425 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -2.56494936 -3.25809654 -2.56494936 -3.25809654
 -3.25809654 -3.25809654]


# 建立伯努利朴素贝叶斯分类器
并进行测试

In [68]:
#建立朴素贝叶斯分类器
def BernoulliNB(test_vec,p0word,p1word,pclass1):
    p0=sum(test_vec*p0word)+np.log(pclass1)
    p1=sum(test_vec*p1word)+np.log(1-pclass1)
    print(sum(test_vec*p0word))
    if p0>p1:
        return 0
    else:
        return 1

## 进行测试

In [69]:
test1=['love','my','dalmtion']

print(f'该文本属于{BernoulliNB(wordvec(wordfeatures,test1),p0word,p1word,p1)}类别')#先把测试文本向量化，再传入分类器中

-4.436751534363128
该文本属于0类别


In [70]:
test2=['stupid','garbage']

print(f'该文本属于{BernoulliNB(wordvec(wordfeatures,test2),p0word,p1word,p1)}类别')

-6.516193076042964
该文本属于1类别


## 直接用模型

In [62]:
from sklearn.naive_bayes import BernoulliNB

In [63]:
a=wordvec(uniqueword(postingList),['stupid','garbage'])
wordvecs

[array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.]),
 array([1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]),
 array([1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.])]

In [64]:
model=MultinomialNB()
model.fit(np.array(wordvecs),np.array(classvec))

MultinomialNB()

In [65]:
model.predict(a.reshape(1,-1))#预测种类

array([1])